# 복합 토픽 모델링 (Combined Topic Modeling)

## 토픽 모델 (Topic Models)
- 비지도 학습 방식으로 토픽 추출 가능

## 문맥을 반영한 토픽 모델 (Contextualized Topic Models, CTM)
- BERT 임베딩의 표현력과 토픽 모델의 비지도 학습의 능력을 결합하여 문서에서 주제를 가져오는 토픽 모델

In [1]:
%%capture
!pip install contextualized-topic-models==2.2.0

In [2]:
%%capture
!pip install pyldavis

In [3]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 1.90 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 44.7 MB/s eta 0:00:00
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2023-01-04 06:48:49--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22cd:e0db, 2406:da00:ff00::

In [4]:
%%capture
!wget https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt

In [7]:
text_file = "2016-10-20.txt"

In [18]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation, bert_embeddings_from_list
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Mecab
from tqdm import tqdm

In [8]:
documents = [line.strip() for line in open(text_file, encoding="utf-8").readlines()]

In [9]:
preprocessed_documents = []

for line in tqdm(documents):
  # 빈 문자열이거나 숫자로만 이루어진 줄은 제외
  if line and not line.replace(' ', '').isdecimal():
    preprocessed_documents.append(line)

100%|██████████| 30091/30091 [00:00<00:00, 169270.74it/s]


In [31]:
preprocessed_documents

['오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다 독자제공 영상 캡처 연합뉴스  서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다  경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다  이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다  성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다  이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다  5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다  용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기  신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다  김 경위는 오패산 터널 입구 오른쪽의 급경사에서 성씨에게 접근하다가 오후 6시 33

In [10]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        word_tokens = self.tagger.morphs(sent)
        result = [word for word in word_tokens if len(word) > 1]
        return result

In [11]:
custom_tokenizer = CustomTokenizer(Mecab())

In [12]:
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)

In [13]:
train_bow_embeddings = vectorizer.fit_transform(preprocessed_documents)

In [14]:
print(train_bow_embeddings.shape)

(27540, 3000)


In [15]:
vocab = vectorizer.get_feature_names()
id2token = {k: v for k, v in zip(range(0, len(vocab)), vocab)}

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [16]:
len(vocab)

3000

In [19]:
train_contextualized_embeddings = bert_embeddings_from_list(preprocessed_documents, \
                                                            "sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens")

Downloading:   0%|          | 0.00/574 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

Batches:   0%|          | 0/138 [00:00<?, ?it/s]

In [21]:
train_contextualized_embeddings.shape

(27540, 768)

In [26]:
qt = TopicModelDataPreparation()
training_dataset = qt.load(train_contextualized_embeddings, train_bow_embeddings, 
                        id2token)

In [28]:
training_dataset

## Combined TM 학습하기

In [29]:
ctm = CombinedTM(bow_size=len(vocab), contextual_size=768, 
                 n_components=50, num_epochs=20)
ctm.fit(training_dataset)

Epoch: [20/20]	 Seen Samples: [550800/550800]	Train Loss: 850.1228141169662	Time: 0:00:07.842002: : 20it [02:45,  8.26s/it]


In [30]:
ctm.get_topics(5)

defaultdict(list,
            {0: ['습니다', '발찌', '경찰', '성병', '사제총'],
             1: ['나노', '양산', '삼성전자', '성능', '구현'],
             2: ['에서', '개구리', '한다', '한복', '으로'],
             3: ['뉴시스', '영상', '언론', '사진', '공감'],
             4: ['공제', '저축', '정산', '분양', '연말'],
             5: ['장관', '회의', '국방', '억제', '외교'],
             6: ['코리아', '여의도', '뉴스', '회견', '일리'],
             7: ['스포츠', '불법행위', '유용', '대통령', '재단'],
             8: ['된다는', '최대한', '움직임', '부여', '통해서'],
             9: ['으로', '에서', '경제', '세금', '다고'],
             10: ['부여', '크기', '눈물', '충분히', '움직임'],
             11: ['제작', '스퀘어', '발표회', '타임', '영등포구'],
             12: ['상선', '채권', '자산', '발행', '국채'],
             13: ['방송', '캡처', '강수지', '전현무', '김국진'],
             14: ['네이버', '대표', '이사', '시장', '한성숙'],
             15: ['연봉제', '중소기업', '성과', '고용', '임금'],
             16: ['계획', '지역', '사업', '시설', '산업'],
             17: ['중구', '2017', '디자인', '열린', '패션'],
             18: ['뉴시스', '영상', '언론', '공감', '화제'],
             19: ['부여', '코리

In [32]:
import pyLDAvis as vis

lda_vis_data = ctm.get_ldavis_data_format(vocab, training_dataset, n_samples=10)

ctm_pd = vis.prepare(**lda_vis_data)
vis.display(ctm_pd)

Output hidden; open in https://colab.research.google.com to view.